##ECE 114 Project: Speaker Region Identification



In this project, we'll train a machine learning algorithm to classify speakers by regional dialect.  We will use speech samples from the Corpus of Regional African American Language (CORAAL - https://oraal.uoregon.edu/coraal) with speakers each belonging to one of five different US cities: 1) Rochester, NY (ROC), 2) Lower East Side, Manhattan, NY (LES), 3) Washington DC (DCB), 4) Princeville, NC (PRV), or 5) Valdosta, GA (VLD).

The project files can be downloaded from [this link](https://ucla.box.com/s/w7s25pffjtjhkphhcb5rpjepi1nt9rg7)

To do this, we will first extract features from the audio files and then train a classifier to predict the city of origin of the utterance's speaker.  The goal is to extract a feature that contains useful information about regional dialect characteristics.

##1. Setting up the data directories and Google Colab

Store a copy of the project files in your google drive.  

Make sure that the 'project_data' folder is stored in the top level of your google drive.  Otherwise, you will need to change the corresponding paths in the remainder of the notebook.

Mount your google drive. This will give this notebook read/write access to data stored in your google drive.  You can either do this in the file browser on the left side of this notebook or by running the code snippet below.

It is recommended that you use your UCLA google account for this project, as it has more storage than a standard google account.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

To run this project on your local system, replace the corresponding file paths to the locations of the project files on your local machine

## 2. Getting familiar with the data


Let's take a moment to understand the data.  The original CORAAL dataset consists of ~85 different speakers, each from one of five cities.  The audio files are names with the convention: DCB_se1_ag1_f_03.  Here, DCB is the city code, se1 denotes the socioeconomic group of the speaker, ag1 denotes the age group of the speaker, f denotes female, and 03 denotes the participant number.  These unique combinations of identifiers mark the speaker.  

The dataset has been preprocessed to only include audio segments greater than 10 seconds in length. there are a number of audio snippets of at least 10sec in length.  Those segments are numbered with the appending tag _seg_number for each segment.

You can also try listening to any segment like this:

In [ ]:
from IPython.display import Audio

sr = 44100

Audio(filename= "../../datasets/", rate=sr)

## 3. Feature Extraction

As a baseline, we will be using the average mfcc value over time from the Librosa Python library. Your job will be to choose better features to improve performance on the test data

We first define a pair of functions to create features and labels for our classification model:


In [ ]:
import librosa
import torchaudio
import numpy as np
from glob import glob
from tqdm import tqdm


def extract_feature(audio_file, n_mfcc=13):

  '''
  Function to extract features from a single audio file given its path
  Modify this function to extract your own custom features
  '''

  audio,fs = torchaudio.load(audio_file)
  audio = audio.numpy().reshape(-1)

  # replace the following features with your own
  mfccs = librosa.feature.mfcc(y=audio, sr=fs, n_mfcc=n_mfcc)
  feat_out = np.mean(mfccs,axis=1)

  lpcs = librsioa.feature.lpc()

  return feat_out


def get_label(file_name):
  '''
  Function to retrieve output labels from filenames
  '''
  if 'ROC' in file_name:
    label=0
  elif 'LES' in file_name:
    label=1
  elif 'DC' in file_name:
    label=2
  elif 'PRV' in file_name:
    label=3
  elif 'VLD' in file_name:
    label=4
  else:
    raise ValueError('invalid file name')
  return label

Let us now call these functions to extract the features and labels from the train directory

Note: This cell might not update the progress bar continuously, but for the baseline MFCC features should take a maximum of 30 minutes to execute

In [ ]:

#First we obtain the list of all files in the train directory
train_files = glob('drive/MyDrive/project_data/train/*.wav')

#Let's sort it so that we're all using the same file list order
#and you can continue processing the features from a given file if it stops
#partway through running
train_files.sort()

train_feat=[]
train_label=[]

for wav in tqdm(train_files):

  train_feat.append(extract_feature(wav))
  train_label.append(get_label(wav))

In [ ]:
#Now we obtain the list of all files in the test directory
test_files = glob('drive/MyDrive/project_data/test/*.wav')

#Similar to above, we sort the files
test_files.sort()

test_feat=[]
test_label=[]

for wav in tqdm(test_files):

  test_feat.append(extract_feature(wav))
  test_label.append(get_label(wav))

## 4. Model Training and Predictions

Now we'll train the backend system to predict the regions from the input features.  We'll use an xgboosted decision tree for this.  An advantage of this model is that we can also parse the decision tree and measure the impact of different features in the end result for explainability

In [ ]:
#Install shap library
# !pip install shap

In [ ]:
import xgboost
import numpy as np
import shap
import pandas as pd

#Format input data

#Edit this variable to create a list that contains your feature names
feat_names=['mfcc_' +str(n) for n in range(len(train_feat[0]))]

train_feat_df = pd.DataFrame(data=np.stack(train_feat), columns=feat_names)
y_train=np.stack(train_label)


test_feat_df = pd.DataFrame(data=np.stack(test_feat), columns=feat_names)
y_test=np.stack(test_label)



#you could just pass in the matrix of features to xgboost
#but it looks prettier in the shap explainer if you format it
#as a dataframe.


model = xgboost.XGBClassifier()
model.fit(train_feat_df,y_train)

print("Train Acc =", np.sum(y_train==model.predict(train_feat_df))/len(y_train))

print("Test Acc =", np.sum(y_test==model.predict(test_feat_df))/len(y_test))



To save a dataframe of features, uncomment and run the following block of code

In [ ]:
# train_feat_df.to_csv('drive/MyDrive/project_data/current_features.csv')

To Load a preexisting dataframe of features (saved from a previous notebook), run the following cell and then train the model

In [ ]:
# train_feat_df = pd.read_csv('drive/MyDrive/project_data/myfeat_train.csv')
# test_feat_df = pd.read_csv('drive/MyDrive/project_data/myfeat_test.csv')

The following cells are to extract features from MATLAB. Ensure that you've run the baseline once before running the cells

Saving list of train and test files

In [ ]:
# Note: we save the list of files to ensure the labels match the utterances
# You can omit this step if you plan on extracting the labels in MATLAB
# But will need to rewrite code in other parts of the notebook

# with open('drive/MyDrive/project_data/train_files.txt', 'w') as f:
#     for line in train_files:
#         f.write(f"{line}\n")

# with open('drive/MyDrive/project_data/test_files.txt', 'w') as f:
#     for line in test_files:
#         f.write(f"{line}\n")


After extracting features using wrapper.m, run the following cell to retrieve a dataframe containing the features

In [ ]:
# train_feat_df = pd.read_csv('drive/MyDrive/project_data/myfeat_train.csv')
# test_feat_df = pd.read_csv('drive/MyDrive/project_data/myfeat_test.csv')

## 5. Interpreting Results and Explainability

To see the impact different features have on the model, we create a plot of the feature importances. The features are listed top to bottom in order of how important they were to the decision.

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train_feat_df)
shap.summary_plot(shap_values, train_feat_df)

And we can see a confusion matrix of the mispredictions

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

confusion_matrix = metrics.confusion_matrix(y_test, model.predict(test_feat_df))
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['ROC','LES','DCB','PRV','VLD'])
cm_display.plot()
plt.show()